In [5]:
!pip install yfinance pandas numpy ta requests beautifulsoup4

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
import requests
from bs4 import BeautifulSoup

In [7]:
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)
    return tickers

In [66]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
def fetch_info():
    try:
        url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0',
            'Accept': 'application/json',
            'Accept-Language': 'en-US,en;q=0.5',
        }
        #  Send GET request
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        #  Get the symbols table
        tables = soup.find_all('table')
        #  #  Convert table to dataframe
        df = pd.read_html(str(tables))[1]
        #  Cleanup
        df.drop(columns=['Notes'], inplace=True)
        return df['Symbol'].to_list()
    except:
        print('Error loading data')
        return None

In [68]:
dow_jones_ticker = fetch_info()

In [72]:
def get_set_data(market = 'SP500', dataset = 'A'):
    
    start_date = '2013-01-01'
    end_date = '2023-12-31'

    if market == 'SP500':
        tickers = get_sp500_tickers()
    
    if market == 'DJones':
        tickers = fetch_info()

    time_index = yf.download('AAPL', start=start_date, end=end_date).index

    if dataset == 'A':
        details_index = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
    if dataset == 'B':
        details_index = ['Open', 'High', 'Low', 'Adj Close', 'Volume', 'RSI', 'MACD', 'SMA5', 'SMA12']
    
    iterables = [time_index, details_index]
    df_index = pd.MultiIndex.from_product(iterables, names=["first", "second"])
    df = pd.DataFrame(index = df_index)

    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date)
        data['RSI'] = ta.momentum.rsi(data['Adj Close'], window=12, fillna=False)
        data['MACD'] = ta.trend.macd_diff(data['Close'], window_fast=12, window_slow=26, fillna=False)
        data['SMA5'] = ta.trend.sma_indicator(data['Close'], window=5, fillna=False)
        data['SMA12'] = ta.trend.sma_indicator(data['Close'], window=12, fillna=False)

        for idx, rows in data.iterrows():
            df.loc[(idx, 'Open'), ticker] = rows['Open']
            df.loc[(idx, 'High'), ticker] = rows['High']
            df.loc[(idx, 'Low'), ticker] = rows['Low']
            df.loc[(idx, 'Adj Close'), ticker] = rows['Adj Close']
            df.loc[(idx, 'Volume'), ticker] = rows['Volume']

            if dataset == 'B':
                df.loc[(idx, 'RSI'), ticker] = rows['RSI']
                df.loc[(idx, 'MACD'), ticker] = rows['MACD']
                df.loc[(idx, 'SMA5'), ticker] = rows['SMA5']
                df.loc[(idx, 'SMA12'), ticker] = rows['SMA12']

    return df

In [94]:
df = get_set_data('DJones', 'A')
df.to_csv('DJones_setA.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [95]:
df = get_set_data('DJones', 'B')
df.to_csv('DJones_setB.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [90]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol=IBM&apikey=demo'
r = requests.get(url)
data = r.json()

print(pd.DataFrame(data['annualEarnings']))

   fiscalDateEnding reportedEPS
0        2023-09-30        3.54
1        2022-12-31        9.12
2        2021-12-31        9.97
3        2020-12-31        8.67
4        2019-12-31       12.81
5        2018-12-31       13.82
6        2017-12-31       13.65
7        2016-12-31        13.6
8        2015-12-31       14.93
9        2014-12-31       16.27
10       2013-12-31       17.03
11       2012-12-31        15.3
12       2011-12-31       13.49
13       2010-12-31       11.58
14       2009-12-31       10.01
15       2008-12-31        8.96
16       2007-12-31        7.19
17       2006-12-31        6.09
18       2005-12-31        5.34
19       2004-12-31        5.07
20       2003-12-31        4.34
21       2002-12-31         3.9
22       2001-12-31        4.44
23       2000-12-31        4.45
24       1999-12-31        3.71
25       1998-12-31        3.32
26       1997-12-31        3.09
27       1996-12-31        2.77


In [92]:
!pip install alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
import matplotlib.pyplot as plt

# Replace with your Alpha Vantage API key
api_key = 'BC7EQJLSFM5GHA1E'

ts = TimeSeries(key=api_key, output_format='pandas')
fd = FundamentalData(key=api_key, output_format='pandas')

# Get daily adjusted close prices

# Get annual EPS
eps_data, _ = fd.get_income_statement_annual(symbol='AAPL')
print(eps_data.columns)
print(eps_data['netIncome'])


Index(['fiscalDateEnding', 'reportedCurrency', 'grossProfit', 'totalRevenue',
       'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome',
       'sellingGeneralAndAdministrative', 'researchAndDevelopment',
       'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome',
       'interestIncome', 'interestExpense', 'nonInterestIncome',
       'otherNonOperatingIncome', 'depreciation',
       'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense',
       'interestAndDebtExpense', 'netIncomeFromContinuingOperations',
       'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome'],
      dtype='object')
date
1970-01-01 00:00:00.000000000    99803000000
1970-01-01 00:00:00.000000001    94680000000
1970-01-01 00:00:00.000000002    57411000000
1970-01-01 00:00:00.000000003    55256000000
1970-01-01 00:00:00.000000004    59531000000
Name: netIncome, dtype: object
